In [16]:
import findspark
findspark.init()
findspark.find()

import os
os.environ["JAVA_HOME"] = "/opt/homebrew/Cellar/openjdk@11/11.0.26/libexec/openjdk.jdk/Contents/Home"

In [17]:
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

In [18]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("YellowTripDataAnalysis").getOrCreate()

df = spark.read.option("header", True).option("inferSchema", True).csv("../data/yellow_tripdata/yellow_tripdata_2015-01.csv")

In [19]:
print(f"Número de columnas: {len(df.columns)}")
print(f"Número de registros: {df.count()}")


Número de columnas: 19


Número de registros: 12748986


In [20]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- RateCodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [21]:
df.dtypes

[('VendorID', 'int'),
 ('tpep_pickup_datetime', 'timestamp'),
 ('tpep_dropoff_datetime', 'timestamp'),
 ('passenger_count', 'int'),
 ('trip_distance', 'double'),
 ('pickup_longitude', 'double'),
 ('pickup_latitude', 'double'),
 ('RateCodeID', 'int'),
 ('store_and_fwd_flag', 'string'),
 ('dropoff_longitude', 'double'),
 ('dropoff_latitude', 'double'),
 ('payment_type', 'int'),
 ('fare_amount', 'double'),
 ('extra', 'double'),
 ('mta_tax', 'double'),
 ('tip_amount', 'double'),
 ('tolls_amount', 'double'),
 ('improvement_surcharge', 'double'),
 ('total_amount', 'double')]

In [22]:
df.describe().show()

+-------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-------------------+---------------------+------------------+
|summary|          VendorID|   passenger_count|     trip_distance|   pickup_longitude|   pickup_latitude|        RateCodeID|store_and_fwd_flag| dropoff_longitude|  dropoff_latitude|      payment_type|       fare_amount|              extra|           mta_tax|        tip_amount|       tolls_amount|improvement_surcharge|      total_amount|
+-------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-------------------+---------------------+---

In [26]:
# Obtenemos el resumen estadístico
describe_df = df.describe()

# Convertimos el resultado de describe a Pandas
describe_pandas = describe_df.toPandas()

# Invertimos el orden de las filas
describe_pandas_inverted = describe_pandas.iloc[::-1]

# Mostramos el DataFrame invertido
print(describe_pandas_inverted)


  summary            VendorID     passenger_count       trip_distance  \
4     max                   2                   9        1.54200045E7   
3     min                   1                   0                 0.0   
2  stddev  0.4995402498256225  1.3379235172874737   9844.094218468374   
1    mean  1.5214373127400094  1.6814908260154964  13.459129611562718   
0   count            12748986            12748986            12748986   

      pickup_longitude     pickup_latitude          RateCodeID  \
4    78.66265106201172  404.70001220703125                  99   
3  -121.92581176757812                 0.0                   1   
2   10.125103592972911     5.5786905190884  0.6732239779497589   
1   -72.56183777902534   39.97282304763482  1.0369007386156044   
0             12748986            12748986            12748986   

  store_and_fwd_flag   dropoff_longitude    dropoff_latitude  \
4                  Y    85.2740249633789   459.5333251953125   
3                  N  -740.166687011

In [28]:
# Obtenemos el resumen estadístico
# describe_df = df.describe()

# Convertimos el resultado de describe a Pandas
describe_pandas = describe_df.toPandas()

# Invertimos el orden de las filas
describe_pandas_inverted = describe_pandas.iloc[::-1]

# Mostramos el DataFrame invertido
print(describe_pandas_inverted)

# Exportamos el DataFrame invertido a Excel
describe_pandas_inverted.to_excel('archivo.xlsx', index=False)

  summary            VendorID     passenger_count       trip_distance  \
4     max                   2                   9        1.54200045E7   
3     min                   1                   0                 0.0   
2  stddev  0.4995402498256225  1.3379235172874737   9844.094218468374   
1    mean  1.5214373127400094  1.6814908260154964  13.459129611562718   
0   count            12748986            12748986            12748986   

      pickup_longitude     pickup_latitude          RateCodeID  \
4    78.66265106201172  404.70001220703125                  99   
3  -121.92581176757812                 0.0                   1   
2   10.125103592972911     5.5786905190884  0.6732239779497589   
1   -72.56183777902534   39.97282304763482  1.0369007386156044   
0             12748986            12748986            12748986   

  store_and_fwd_flag   dropoff_longitude    dropoff_latitude  \
4                  Y    85.2740249633789   459.5333251953125   
3                  N  -740.166687011

In [31]:
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.types import NumericType

# Detectamos si la columna es numérica
def count_nulls(df):
    exprs = []
    for c in df.columns:
        if isinstance(df.schema[c].dataType, NumericType):
            exprs.append(count(when(col(c).isNull() | isnan(c), c)).alias(c))
        else:
            exprs.append(count(when(col(c).isNull(), c)).alias(c))
    return df.select(exprs)

# Ahora usamos nuestra función
null_counts = count_nulls(df)

# Lo pasamos a pandas y acomodamos bonito
null_counts = null_counts.toPandas().T
null_counts.columns = ['null_count']
null_counts = null_counts.sort_values('null_count', ascending=False)

# Invertimos el orden de las filas
describe_pandas_inverted = null_counts

# Mostramos el DataFrame invertido
print(describe_pandas_inverted)

# Exportamos el DataFrame invertido a Excel
describe_pandas_inverted.to_excel('archivo.xlsx', index=False)



print(null_counts)


                       null_count
improvement_surcharge           3
VendorID                        0
dropoff_latitude                0
tolls_amount                    0
tip_amount                      0
mta_tax                         0
extra                           0
fare_amount                     0
payment_type                    0
dropoff_longitude               0
tpep_pickup_datetime            0
store_and_fwd_flag              0
RateCodeID                      0
pickup_latitude                 0
pickup_longitude                0
trip_distance                   0
passenger_count                 0
tpep_dropoff_datetime           0
total_amount                    0
                       null_count
improvement_surcharge           3
VendorID                        0
dropoff_latitude                0
tolls_amount                    0
tip_amount                      0
mta_tax                         0
extra                           0
fare_amount                     0
payment_type  

25/04/28 02:21:02 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 946729 ms exceeds timeout 120000 ms
25/04/28 02:21:02 WARN SparkContext: Killing executors is not supported by current scheduler.
25/04/28 02:21:10 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [ ]:
# Convertimos el resultado de describe a Pandas
describe_pandas = describe_df.toPandas()

# Invertimos el orden de las filas
describe_pandas_inverted = describe_pandas.iloc[::-1]

# Mostramos el DataFrame invertido
print(describe_pandas_inverted)

# Exportamos el DataFrame invertido a Excel
describe_pandas_inverted.to_excel('archivo.xlsx', index=False)